In [ ]:
"""
Options Arbitrage Tracker 
Tracks options for arbitrage opportunities using Black-Scholes and GBM models.
Generates BUY/SELL signals based on a 20% price difference threshold.
Logs trades to CSV for future backtesting. Version 0.1 - Live Tracker Only.
"""

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# Stocks History 
Stocks_Symbol = 'TSLA'
df_Stocks = yf.download(Stocks_Symbol, start = '2022-01-01', end= pd.Timestamp.today().strftime('%Y-%m-%d'))[['Close']]
df_Stocks.columns = ['Close']

# Options History 
Options = yf.Ticker(Stocks_Symbol)
Options_Expiration = Options.options
Options_Chain = Options.option_chain(Options_Expiration[1]) # 0 - soonest expiry , 1 - second soonest expiry
Options_Calls = Options_Chain.calls.copy()
Options_Puts = Options_Chain.puts.copy()
Options_Calls['Option_Type'] = "Call"
Options_Puts['Option_Type'] = "Put"
Options_Calls['Expiration'] = pd.to_datetime("20" + Options_Calls['contractSymbol'].str.extract(r'(\d{6})')[0], format = '%Y%m%d')
Options_Puts['Expiration'] = pd.to_datetime("20"+ Options_Puts['contractSymbol'].str.extract(r'(\d{6})')[0], format = '%Y%m%d')
Options_Calls = Options_Calls[Options_Calls['impliedVolatility'] > 0.1].reset_index(drop=True)         # Implied Volatility : Given by options data
Options_Puts = Options_Puts[Options_Puts['impliedVolatility'] > 0.1].reset_index(drop=True)
df_Options = pd.concat([Options_Calls, Options_Puts])


# Options Chain Based On ATM Option Price 
Options_ATM_Calls = Options_Calls.iloc[(Options_Calls['strike']-df_Stocks['Close'].iloc[-1]).abs().argmin()]
Options_ATM_Puts = Options_Puts.iloc[(Options_Puts['strike']-df_Stocks['Close'].iloc[-1]).abs().argmin()]

# Project Stock Price
Historical_Volatility_Window = 30
Annual_Drift_Window = 252
Risk_Free_Rate = 0.05

# Historical Volatility = Standard Deviation on Returns (Annualized and Percentage)
# Formula: Std(In(Price_0 / Price_0-1)) *  √252 * 100
df_Stocks['Historical_Volatility'] = ((np.log(df_Stocks['Close']/df_Stocks['Close'].shift(1)).rolling(Historical_Volatility_Window).std()) * np.sqrt(252)).fillna(0) 
# Annual Drift = Mean Log Returns x 252 (Annualized)
# Formula: Mean(In(Price_0 / Price_0-1)) *  252
df_Stocks['Annual_Drift'] = (((np.log(df_Stocks['Close']/df_Stocks['Close'].shift(1))).rolling(Annual_Drift_Window).mean()) * 252).fillna(0)

# Option Pricing (Black-Scholes)
Black_Scholes_T = ((Options_ATM_Calls['Expiration'] - pd.Timestamp.today().normalize()).days)/252
Current_Call_Price = Options_ATM_Calls['lastPrice']
Current_Put_Price = Options_ATM_Puts['lastPrice']
Current_Stock_Price = df_Stocks['Close'].iloc[-1]

# Black-Scholes Functions
# Formula: Call_Price = S_0 * N(D1) - K*exp*(-rT) * N(D2)
# Formula: Put_Price = K*exp*(-rT) * N(-D2) - S_0 * N(-D1)
# Formula: D1 = In(S_0/K)+(R+(0.5*σ**2))*T / (σ*√T)
# Formula: D2 = D1 - (σ*√T)
def Black_Scholes_Calls_Price (Sigma, Current_Stock_Price , Return_Delta = False , Return_Delta_And_Price=False):
    Calls_Black_Scholes_D1 = (np.log(Current_Stock_Price/Options_ATM_Calls['strike']) + (Risk_Free_Rate + 0.5*(Sigma)**2)*Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    Calls_Black_Scholes_D2 = Calls_Black_Scholes_D1 - (Sigma*np.sqrt(Black_Scholes_T))
    Calls_Black_Scholes_Price = Current_Stock_Price*norm.cdf(Calls_Black_Scholes_D1) - (Options_ATM_Calls['strike']*np.exp(-Risk_Free_Rate*Black_Scholes_T)*norm.cdf(Calls_Black_Scholes_D2))
    Calls_Delta = norm.cdf(Calls_Black_Scholes_D1)

    if Return_Delta_And_Price:
        return Calls_Black_Scholes_Price , Calls_Delta
    if Return_Delta:
        return Calls_Black_Scholes_Price - Options_ATM_Calls['lastPrice']
    return Calls_Black_Scholes_Price

def Black_Scholes_Puts_Price (Sigma, Current_Stock_Price, Return_Delta = False , Return_Delta_And_Price=False):
    Puts_Black_Scholes_D1 = (np.log(Current_Stock_Price/Options_ATM_Puts['strike']) + (Risk_Free_Rate + 0.5*(Sigma)**2)*Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    Puts_Black_Scholes_D2 = Puts_Black_Scholes_D1 - (Sigma*np.sqrt(Black_Scholes_T))
    Puts_Black_Scholes_Price = (Options_ATM_Puts['strike']*np.exp(-Risk_Free_Rate*Black_Scholes_T)*norm.cdf(-Puts_Black_Scholes_D2)) - Current_Stock_Price*norm.cdf(-Puts_Black_Scholes_D1)
    Puts_Delta = norm.cdf(Puts_Black_Scholes_D1)
    if Return_Delta_And_Price:
        return Puts_Black_Scholes_Price , Puts_Delta
    if Return_Delta:
        return Puts_Black_Scholes_Price - Options_ATM_Puts['lastPrice']
    return Puts_Black_Scholes_Price

def Vega_Calls (Sigma):
    Calls_Black_Scholes_D1 = (np.log(Current_Stock_Price/Options_ATM_Calls['strike']) + (Risk_Free_Rate + 0.5*(Sigma)**2)*Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    return Current_Stock_Price * np.sqrt(Black_Scholes_T) * norm.pdf(Calls_Black_Scholes_D1)

def Vega_Puts (Sigma):
    Puts_Black_Scholes_D1 = (np.log(Current_Stock_Price/Options_ATM_Puts['strike']) + (Risk_Free_Rate + 0.5*(Sigma)**2)*Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    return Current_Stock_Price * np.sqrt(Black_Scholes_T) * norm.pdf(Puts_Black_Scholes_D1)


# Newton-Raphson Implied Volatility 
# Formula: (Current Sigma,σ) - ((Black_Scholes_Price - Current_Options_Price (Last traded)) / Black_Scholes_D1 (Vega)) 
Sigma_Calls = df_Stocks['Historical_Volatility'].iloc[-1]
Sigma_Puts = df_Stocks['Historical_Volatility'].iloc[-1]

for i in range (100):
    Sigma_Delta = Black_Scholes_Calls_Price (Sigma_Calls, Current_Stock_Price, Return_Delta = True )
    Vega_Delta = Vega_Calls(Sigma_Calls)
    if abs(Sigma_Delta) < 0.0001:
        break
    Sigma_Calls = Sigma_Calls - Sigma_Delta / Vega_Delta
Newton_Raphson_Implied_Volatility_Calls = Sigma_Calls

for i in range (100):
    Sigma_Delta = Black_Scholes_Puts_Price (Sigma_Puts, Current_Stock_Price, Return_Delta = True )
    Vega_Delta = Vega_Puts(Sigma_Puts)
    if abs(Sigma_Delta) < 0.0001:
        break
    Sigma_Puts = Sigma_Puts - Sigma_Delta / Vega_Delta
Newton_Raphson_Implied_Volatility_Puts = Sigma_Puts

# Geometric Brownian Motion (GBM) 
# Formula: S_T = S_0 * exp((r - 0.5σ²)T + σ√T * Z)
# Formula: Z = N(0,1) * √T
# Wiener Process (Random Noise) + Monte Carlo Simulation
Monte_Carlo_Simulations = 100_000
np.random.seed(42)
GBM_T = ((Options_ATM_Calls['Expiration'] - pd.Timestamp.today().normalize()).days)/252
Wiener_Process = np.random.normal(0,1,Monte_Carlo_Simulations) * np.sqrt (GBM_T)

# GBM (Historical Volatility)
Stock_Price_Projection_GBM_Historical_Volatility = df_Stocks['Close'].iloc[-1] * np.exp (((df_Stocks['Annual_Drift'].iloc[-1]-(0.5*((df_Stocks['Historical_Volatility'].iloc[-1])**2)))*GBM_T) + ((df_Stocks['Historical_Volatility'].iloc[-1])*(Wiener_Process)))

# GBM (YFinance Volatility)
Stock_Price_Projection_GBM_YFinance_Calls = df_Stocks['Close'].iloc[-1] * np.exp (((df_Stocks['Annual_Drift'].iloc[-1]-(0.5*((Options_ATM_Calls['impliedVolatility'])**2)))*GBM_T) + ((Options_ATM_Calls['impliedVolatility'])*(Wiener_Process)))
Stock_Price_Projection_GBM_YFinance_Puts = df_Stocks['Close'].iloc[-1] * np.exp (((df_Stocks['Annual_Drift'].iloc[-1]-(0.5*((Options_ATM_Puts['impliedVolatility'])**2)))*GBM_T) + ((Options_ATM_Puts['impliedVolatility'])*(Wiener_Process)))

# GBM (Newton Raphson Volatility)
Stock_Price_Projection_GBM_Newton_Raphson_Calls = df_Stocks['Close'].iloc[-1] * np.exp (((df_Stocks['Annual_Drift'].iloc[-1]-(0.5*((Newton_Raphson_Implied_Volatility_Calls)**2)))*GBM_T) + ((Newton_Raphson_Implied_Volatility_Calls)*(Wiener_Process)))
Stock_Price_Projection_GBM_Newton_Raphson_Puts = df_Stocks['Close'].iloc[-1] * np.exp (((df_Stocks['Annual_Drift'].iloc[-1]-(0.5*((Newton_Raphson_Implied_Volatility_Puts)**2)))*GBM_T) + ((Newton_Raphson_Implied_Volatility_Puts)*(Wiener_Process)))

# Option Price (GBM)
# Obtaining Option prices from future stock price (GBM)
# Formula: Call_Price = exp*(-rT) * Mean (np.Maximum (S_T - K,0))
# Formula: Put_Price = exp*(-rT) * Mean (np.Maximum (K - S_T,0))

# Option Price GBM (Historical Volatility)
Calls_GBM_Historical_Volatility_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Stock_Price_Projection_GBM_Historical_Volatility-Options_ATM_Calls['strike'],0))
Puts_GBM_Historical_Volatility_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Options_ATM_Puts['strike']-Stock_Price_Projection_GBM_Historical_Volatility,0))

# Option Price GBM (YFinance Volatility)
Calls_GBM_YFinance_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Stock_Price_Projection_GBM_YFinance_Calls-Options_ATM_Calls['strike'],0))
Puts_GBM_YFinance_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Options_ATM_Puts['strike']-Stock_Price_Projection_GBM_YFinance_Puts,0))

# Option Price GBM (Newton Raphson)
Calls_GBM_Newton_Raphson_Volatility_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Stock_Price_Projection_GBM_Newton_Raphson_Calls-Options_ATM_Calls['strike'],0))
Puts_GBM_Newton_Raphson_Volatility_Price = np.exp(-Risk_Free_Rate*GBM_T)*np.mean(np.maximum(Options_ATM_Puts['strike']-Stock_Price_Projection_GBM_Newton_Raphson_Puts,0))

# Entry, TP & SL Logic
Entry_Logic = 0.20 
BS_Calls_Price_Difference = (Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])-Options_ATM_Calls['lastPrice'])/Options_ATM_Calls['lastPrice']
BS_Puts_Price_Difference = (Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])-Options_ATM_Puts['lastPrice'])/Options_ATM_Puts['lastPrice']
GBM_Calls_Price_Difference = (Calls_GBM_Historical_Volatility_Price-Options_ATM_Calls['lastPrice'])/Options_ATM_Calls['lastPrice']
GBM_Puts_Price_Difference = (Puts_GBM_Historical_Volatility_Price -Options_ATM_Puts['lastPrice'])/Options_ATM_Puts['lastPrice']
Calls_SL = Options_ATM_Calls['lastPrice'] * 0.7
Puts_SL = Options_ATM_Puts['lastPrice'] * 0.7

# Prints : Basis
print(f"{Stocks_Symbol} -> Closing Price : {df_Stocks['Close'].iloc[-1]:.2f} | Annual Drift : {(df_Stocks['Annual_Drift'].iloc[-1]) * 100:.2f}% | Calculated Volatility (Newton-Raphson) -> Calls: {Newton_Raphson_Implied_Volatility_Calls*100:.2f} % ; Puts: {Newton_Raphson_Implied_Volatility_Puts*100:.2f}%")
print(f"Expiry: {Options_ATM_Calls['Expiration'].strftime('%Y-%m-%d')} | Strike Price -> Calls : {Options_ATM_Calls['strike']} ; Puts : {Options_ATM_Puts['strike']} | Historical Volatility : {df_Stocks['Historical_Volatility'].iloc[-1]*100:.2f}% | Implied Volatility (YFinance) -> Calls : {Options_ATM_Calls['impliedVolatility']*100:.2f}% ; Puts : {Options_ATM_Puts['impliedVolatility']*100:.2f}%")

# Trade DataFrame for Backtesting Prep
Trade_Data = {
    'Date': [pd.Timestamp.today().strftime('%Y-%m-%d')],
    'Stock_Price': [df_Stocks['Close'].iloc[-1]],
    'Call_Strike': [Options_ATM_Calls['strike']],
    'Put_Strike': [Options_ATM_Puts['strike']],
    'Call_Market_Price': [Options_ATM_Calls['lastPrice']],
    'Put_Market_Price': [Options_ATM_Puts['lastPrice']],
    'BS_Call_Theoretical': [Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])],
    'BS_Put_Theoretical': [Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])],
    'GBM_Call_Theoretical': [Calls_GBM_Historical_Volatility_Price],
    'GBM_Put_Theoretical': [Puts_GBM_Historical_Volatility_Price],
    'BS_Call_Diff': [BS_Calls_Price_Difference],
    'BS_Put_Diff': [BS_Puts_Price_Difference],
    'GBM_Call_Diff': [GBM_Calls_Price_Difference],
    'GBM_Put_Diff': [GBM_Puts_Price_Difference]
}
df_Trades = pd.DataFrame(Trade_Data)

# Trades
print("\nTrade Criteria : Volatility Diff < 20%; SL 30% ")
print("Black Scholes Model (Historical Volatility)")
if BS_Calls_Price_Difference > Entry_Logic:
    call_sl = Options_ATM_Calls['lastPrice'] * 0.7
    df_Trades['Call_Action'] = 'BUY'
    df_Trades['Call_SL'] = call_sl
    df_Trades['Call_TP'] = Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])
    print(f"BUY Call: Market ${Options_ATM_Calls['lastPrice']:.2f} < Theoretical ${Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f} (+{BS_Calls_Price_Difference*100:.2f}%)")
    print(f"Call SL: ${call_sl:.2f} | TP: ${Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f}")
elif BS_Calls_Price_Difference < -Entry_Logic:
    call_sl = Options_ATM_Calls['lastPrice'] * 1.3
    df_Trades['Call_Action'] = 'SELL'
    df_Trades['Call_SL'] = call_sl
    df_Trades['Call_TP'] = Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])
    print(f"SELL Call: Market ${Options_ATM_Calls['lastPrice']:.2f} > Theoretical ${Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f} (-{BS_Calls_Price_Difference*100:.2f}%)")
    print(f"Call SL: ${call_sl:.2f} | TP: ${Black_Scholes_Calls_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f}")
else:
    print(f"HOLD Call: Price difference {BS_Calls_Price_Difference*100:.2f}% within 20% threshold")

if BS_Puts_Price_Difference > Entry_Logic:
    put_sl = Options_ATM_Puts['lastPrice'] * 0.7
    df_Trades['Put_Action'] = 'BUY'
    df_Trades['Put_SL'] = put_sl
    df_Trades['Put_TP'] = Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])
    print(f"BUY Put: Market ${Options_ATM_Puts['lastPrice']:.2f} < Theoretical ${Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f} (+{BS_Puts_Price_Difference*100:.2f}%)")
    print(f"Put SL: ${put_sl:.2f} | TP: ${Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f}")
elif BS_Puts_Price_Difference < -Entry_Logic:
    put_sl = Options_ATM_Puts['lastPrice'] * 1.3
    df_Trades['Put_Action'] = 'SELL'
    df_Trades['Put_SL'] = put_sl
    df_Trades['Put_TP'] = Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1])
    print(f"SELL Put: Market ${Options_ATM_Puts['lastPrice']:.2f} > Theoretical ${Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f} (-{BS_Puts_Price_Difference*100:.2f}%)")
    print(f"Put SL: ${put_sl:.2f} | TP: ${Black_Scholes_Puts_Price(df_Stocks['Historical_Volatility'].iloc[-1], df_Stocks['Close'].iloc[-1]):.2f}")
else:
    print(f"HOLD Put: Price difference {BS_Puts_Price_Difference*100:.2f}% within 20% threshold")

print("\nGeometric Brownian Motion Model (Historical Volatility)")
if GBM_Calls_Price_Difference > Entry_Logic:
    call_sl = Options_ATM_Calls['lastPrice'] * 0.7
    df_Trades['Call_Action'] = 'BUY'
    df_Trades['Call_SL'] = call_sl
    df_Trades['Call_TP'] = Calls_GBM_Historical_Volatility_Price
    print(f"BUY Call: Market ${Options_ATM_Calls['lastPrice']:.2f} < Theoretical ${Calls_GBM_Historical_Volatility_Price:.2f} (+{GBM_Calls_Price_Difference*100:.2f})")
    print(f"Call SL: ${call_sl:.2f} | TP: ${Calls_GBM_Historical_Volatility_Price:.2f}")
elif GBM_Calls_Price_Difference < -Entry_Logic:
    call_sl = Options_ATM_Calls['lastPrice'] * 1.3
    df_Trades['Call_Action'] = 'SELL'
    df_Trades['Call_SL'] = call_sl
    df_Trades['Call_TP'] = Calls_GBM_Historical_Volatility_Price
    print(f"SELL Call: Market ${Options_ATM_Calls['lastPrice']:.2f} > Theoretical ${Calls_GBM_Historical_Volatility_Price:.2f} (-{GBM_Calls_Price_Difference*100:.2f}%)")
    print(f"Call SL: ${call_sl:.2f} | TP: ${Calls_GBM_Historical_Volatility_Price:.2f}")
else:
    print(f"HOLD Call: Price difference {GBM_Calls_Price_Difference*100:.2f}% within 20% threshold")

if GBM_Puts_Price_Difference > Entry_Logic:
    put_sl = Options_ATM_Puts['lastPrice'] * 0.7
    df_Trades['Put_Action'] = 'BUY'
    df_Trades['Put_SL'] = put_sl
    df_Trades['Put_TP'] = Puts_GBM_Historical_Volatility_Price
    print(f"BUY Put: Market ${Options_ATM_Puts['lastPrice']:.2f} < Theoretical ${Puts_GBM_Historical_Volatility_Price:.2f} (+{GBM_Puts_Price_Difference*100:.2f}%)")
    print(f"Put SL: ${put_sl:.2f} | TP: ${Puts_GBM_Historical_Volatility_Price:.2f}")
elif GBM_Puts_Price_Difference < -Entry_Logic:
    put_sl = Options_ATM_Puts['lastPrice'] * 1.3
    df_Trades['Put_Action'] = 'SELL'
    df_Trades['Put_SL'] = put_sl
    df_Trades['Put_TP'] = Puts_GBM_Historical_Volatility_Price
    print(f"SELL Put: Market ${Options_ATM_Puts['lastPrice']:.2f} > Theoretical ${Puts_GBM_Historical_Volatility_Price:.2f} (-{GBM_Puts_Price_Difference*100:.2f}%)")
    print(f"Put SL: ${put_sl:.2f} | TP: ${Puts_GBM_Historical_Volatility_Price:.2f}")
else:
    print(f"HOLD Put: Price difference {GBM_Puts_Price_Difference*100:.2f}% within 20% threshold")

# WorkSheet
# df_Options.to_csv(r"...\Options_Arbitrage_Tracker_Options_History.csv", index=True)
# df_Stocks.to_csv(r"...\Options_Arbitrage_Tracker_Stocks_Worksheet.csv", index=True)
# df_Trades.to_csv(r"...\Options_Arbitrage_Tracker_Trade_Log.csv", index=False)



[*********************100%***********************]  1 of 1 completed


TSLA -> Closing Price : 235.86 | Annual Drift : 36.60% | Calculated Volatility (Newton-Raphson) -> Calls: 60.26 % ; Puts: 65.85%
Expiry: 2025-03-28 | Strike Price -> Calls : 252.5 ; Puts : 220.0 | Historical Volatility : 77.81% | Implied Volatility (YFinance) -> Calls : 12.50% ; Puts : 12.50%

Trade Criteria : Volatility Diff < 20%; SL 30% 
Black Scholes Model (Historical Volatility)
BUY Call: Market $4.27 < Theoretical $6.89 (+61.29%)
Call SL: $2.99 | TP: $6.89
BUY Put: Market $4.45 < Theoretical $6.10 (+37.16%)
Put SL: $3.11 | TP: $6.10

Geometric Brownian Motion Model (Historical Volatility)
BUY Call: Market $4.27 < Theoretical $7.76 (+81.71)
Call SL: $2.99 | TP: $7.76
BUY Put: Market $4.45 < Theoretical $5.47 (+22.87%)
Put SL: $3.11 | TP: $5.47
